# Horse Feature DataFrame Generation with Train-Test Split

This notebook contains functions to load and analyze ground truth data for horse detection. It processes GeoJSON data and associated orthomosaic tiles to create a feature dataset suitable for machine learning. The notebook also implements a train-test split and exports the data in a format compatible with Hugging Face datasets.

## Setup and Imports

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from pathlib import Path
import os
from io import BytesIO
from tqdm import tqdm
import json
from sklearn.cluster import KMeans
import random
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Constants for data directories relative to project root
DATA_DIR = "../data"
PROCESSED_DIR = os.path.join(DATA_DIR, "processed")
SPLITS_DIR = os.path.join(PROCESSED_DIR, "splits")
HF_DATA_DIR = os.path.join(DATA_DIR, "hf_dataset")

# Create necessary directories
os.makedirs(SPLITS_DIR, exist_ok=True)
os.makedirs(HF_DATA_DIR, exist_ok=True)

## Data Loading Functions

These functions handle loading and analyzing the ground truth data from GeoJSON files. The data contains point features representing horse presence/absence observations.

### `load_ground_truth`
Loads ground truth data from a GeoJSON file into a GeoDataFrame. The function includes basic error handling for file existence.

In [ ]:
def load_ground_truth(
    filepath: str = "../data/vector/groundtruth.geojson",
) -> gpd.GeoDataFrame:
    """Load ground truth data from a GeoJSON file."""
    path = Path(filepath)
    if not path.exists():
        raise FileNotFoundError(f"File not found: {filepath}")
    return gpd.read_file(filepath)

### `get_point_info`
Extracts basic information about point features from the GeoDataFrame, including:
- Total number of points
- Available columns
- Spatial bounds of the data

In [ ]:
def get_point_info(gdf: gpd.GeoDataFrame) -> dict:
    """Get basic information about point features."""
    points = gdf[gdf.geometry.type == "Point"]
    return {
        "total_points": len(points),
        "columns": list(points.columns),
        "bounds": points.total_bounds.tolist(),
    }

In [ ]:
# Load the ground truth data
gdf = load_ground_truth()

# Get and display basic information
info = get_point_info(gdf)
print("Ground Truth Information:")
for key, value in info.items():
    print(f"{key}: {value}")

# Display the first few rows of the GeoDataFrame
print("\nFirst few rows of the ground truth data:")
display(gdf.head())

## Tile Processing Functions

This section contains functions for processing and encoding GeoTIFF tiles. These functions handle the conversion of image data into a format suitable for machine learning.

### `encode_tile`
Reads a GeoTIFF tile and encodes it as compressed bytes. The function:
- Loads the raster data using rasterio
- Preserves both pixel data and metadata
- Compresses the data using numpy's savez_compressed
- Returns the encoded data as bytes

In [ ]:
def encode_tile(tile_path: str) -> bytes:
    """Read a GeoTIFF tile and encode it as bytes."""
    if not os.path.exists(tile_path):
        raise FileNotFoundError(f"Tile not found: {tile_path}")

    with rasterio.open(tile_path) as src:
        data = src.read()
        bio = BytesIO()
        np.savez_compressed(bio, data=data, **src.meta)
        bio.seek(0)
        return bio.read()

## Feature DataFrame Creation

This section handles the creation of a comprehensive DataFrame that combines ground truth data with orthomosaic information. The process involves:
- Converting GeoDataFrame to standard DataFrame
- Processing orthomosaic dates and directories
- Creating feature rows based on temporal relationships

### `create_feature_dataframe`
Creates a DataFrame by combining ground truth points with orthomosaic information. Key features:
- Uses all available orthomosaics for both presence and absence points
- Generates tile paths for each point-orthomosaic combination
- Adds an `observation_offset` column, which is a signed integer representing the number of days between the orthomosaic date and the ground truth date
  - Positive values indicate the orthomosaic was captured after the ground truth observation
  - Negative values indicate the orthomosaic was captured before the ground truth observation

In [ ]:
def create_feature_dataframe(
    gdf: gpd.GeoDataFrame, tiles_dir: str = "../data/raster/tiles"
) -> gpd.GeoDataFrame:
    # Retain the geometry column
    df = gdf.copy()

    # Get and sort orthomosaics
    orthomosaics = sorted(
        [
            d
            for d in os.listdir(tiles_dir)
            if os.path.isdir(os.path.join(tiles_dir, d)) and not d.startswith(".")
        ]
    )

    # Convert dates
    ortho_dates = pd.to_datetime(
        [d.split("_")[0] for d in orthomosaics], format="%y%m%d", utc=True
    ).tz_localize(None)

    # Create rows for each orthomosaic and calculate observation offset
    rows = []
    for idx, row in df.iterrows():
        row_date = pd.to_datetime(row["Datetime"]).tz_localize(None)
        # Get the geometry for this row
        geometry = row.geometry

        for ortho, ortho_date in zip(orthomosaics, ortho_dates):
            new_row = row.to_dict()
            new_row["orthomosaic"] = ortho
            new_row["tile_path"] = os.path.join(
                tiles_dir,
                ortho,
                "presence" if row["Presence"] == 1 else "absence",
                f"{int(row['idx']):04d}.tif",
            )
            # Calculate the observation offset in days as a signed integer
            # Positive value means orthomosaic was captured after the ground truth date
            # Negative value means orthomosaic was captured before the ground truth date
            new_row["observation_offset"] = (ortho_date - row_date).days
            # Add the geometry to each new row
            new_row["geometry"] = geometry
            rows.append(new_row)

    # Create a GeoDataFrame from the rows
    feature_gdf = gpd.GeoDataFrame(rows)
    
    # Remove the 'tile_path' column
    feature_gdf.drop(columns=['tile_path'], inplace=True)
    
    return feature_gdf

## Train-Test Split Functions

This section contains functions for creating a stratified train-test split of the data. The split is designed to maintain the distribution of presence/absence points while ensuring spatial and temporal representativeness.

### `filter_absences_near_presences`
Filters out absence points that are within a specified distance of any presence point. This helps ensure that absence points are truly representative of areas without horses, rather than being near-misses of presence points.

In [ ]:
def filter_absences_near_presences(gdf, buffer_distance=2):
    """
    Filter out absence points that are within buffer_distance meters of any presence point.
    
    Args:
        gdf: GeoDataFrame containing both presence and absence points
        buffer_distance: Distance in meters to buffer around presence points
        
    Returns:
        GeoDataFrame containing only valid absence points
    """
    # Split into presence and absence
    presence_gdf = gdf[gdf['Presence'] == 1]
    absence_gdf = gdf[gdf['Presence'] == 0]
    
    print(f"Original absence points: {len(absence_gdf)}")
    
    # Create buffers around presence points
    presence_buffers = presence_gdf.copy()
    presence_buffers.geometry = presence_gdf.geometry.buffer(buffer_distance)
    
    # Find absences that intersect with any presence buffer
    valid_absences = absence_gdf.copy()
    to_remove = []
    
    for idx, absence in tqdm(absence_gdf.iterrows(), total=len(absence_gdf), desc="Filtering absences"):
        for _, buffer in presence_buffers.iterrows():
            if absence.geometry.intersects(buffer.geometry):
                to_remove.append(idx)
                break
    
    valid_absences = valid_absences.drop(to_remove)
    print(f"Valid absence points (after removing those within {buffer_distance}m of presences): {len(valid_absences)}")
    
    return valid_absences

### `create_spatial_blocks`
Creates spatial blocks using K-means clustering on point coordinates. This function helps ensure that the train-test split is spatially representative by grouping points based on their geographic proximity.

In [ ]:
def create_spatial_blocks(gdf, n_clusters=10):
    """
    Create spatial blocks using K-means clustering on point coordinates.
    
    Args:
        gdf: GeoDataFrame containing points
        n_clusters: Number of spatial blocks to create
        
    Returns:
        GeoDataFrame with an additional 'spatial_block' column
    """
    # Extract coordinates for clustering
    coords = np.array([(p.x, p.y) for p in gdf.geometry])
    
    # Apply K-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=RANDOM_SEED)
    clusters = kmeans.fit_predict(coords)
    
    # Add cluster labels to the GeoDataFrame
    result_gdf = gdf.copy()
    result_gdf['spatial_block'] = clusters
    
    return result_gdf

### `create_train_test_split`
Creates a train-test split according to specific criteria:
1. For absences: exclude any absence found within 2 meters of a presence (at any time point), then create spatial blocks using K-means clustering and sample 20% randomly from each block
2. For presences: group by zone and period, sampling 20% randomly from each group

This approach ensures that the test set is representative of the full dataset across both spatial and temporal dimensions.

In [ ]:
def create_train_test_split(gdf, test_size=0.2):
    """
    Create train/test split according to specific criteria:
    1. For absences: exclude any absence found within 2 meters of a presence,
       then create spatial blocks and sample test_size from each block
    2. For presences: group by zone and period, sampling test_size from each group
    
    Args:
        gdf: GeoDataFrame containing both presence and absence points
        test_size: Proportion of data to include in the test split
        
    Returns:
        train_gdf, test_gdf: GeoDataFrames for training and testing
    """
    # Split into presence and absence
    presence_gdf = gdf[gdf['Presence'] == 1].copy()
    absence_gdf = gdf[gdf['Presence'] == 0].copy()
    
    # Filter absences near presences
    valid_absences = filter_absences_near_presences(gdf)
    
    # Create spatial blocks for absences
    absence_blocks = create_spatial_blocks(valid_absences)
    
    # Initialize lists to store indices
    presence_test_indices = []
    presence_train_indices = []
    absence_test_indices = []
    absence_train_indices = []
    
    # Split presences by zone and period
    for (zone, period), group in presence_gdf.groupby(['Zone', 'Period']):
        if len(group) > 0:
            # Calculate number of test samples for this group
            n_test = max(1, int(len(group) * test_size))
            
            # Randomly select indices for test set
            group_indices = group.index.tolist()
            test_idx = random.sample(group_indices, n_test)
            train_idx = [idx for idx in group_indices if idx not in test_idx]
            
            # Add to master lists
            presence_test_indices.extend(test_idx)
            presence_train_indices.extend(train_idx)
    
    # Split absences by spatial block
    for block, group in absence_blocks.groupby('spatial_block'):
        if len(group) > 0:
            # Calculate number of test samples for this block
            n_test = max(1, int(len(group) * test_size))
            
            # Randomly select indices for test set
            group_indices = group.index.tolist()
            test_idx = random.sample(group_indices, n_test)
            train_idx = [idx for idx in group_indices if idx not in test_idx]
            
            # Add to master lists
            absence_test_indices.extend(test_idx)
            absence_train_indices.extend(train_idx)
    
    # Combine test and train indices
    test_indices = presence_test_indices + absence_test_indices
    train_indices = presence_train_indices + absence_train_indices
    
    # Create test and train dataframes
    test_gdf = gdf.loc[test_indices].copy()
    train_gdf = gdf.loc[train_indices].copy()
    
    # Print split statistics
    print(f"Total points: {len(gdf)}")
    print(f"Training points: {len(train_gdf)} ({len(train_gdf)/len(gdf)*100:.1f}%)")
    print(f"Testing points: {len(test_gdf)} ({len(test_gdf)/len(gdf)*100:.1f}%)")
    
    # Check class distribution in each split
    print("\nClass distribution in training set:")
    print(train_gdf['Presence'].value_counts(normalize=True) * 100)
    
    print("\nClass distribution in test set:")
    print(test_gdf['Presence'].value_counts(normalize=True) * 100)
    
    return train_gdf, test_gdf, train_indices, test_indices

### `visualize_train_test_split`
Creates a visualization of the train-test split, showing the spatial distribution of presence and absence points in both the training and testing sets. This visualization helps to verify that the spatial stratification is working correctly and that the test set is representative of the overall data distribution.

In [ ]:
def visualize_train_test_split(train_gdf, test_gdf, figsize=(14, 12)):
    """
    Visualize the train-test split on a map.
    
    Args:
        train_gdf: GeoDataFrame containing training points
        test_gdf: GeoDataFrame containing testing points
        figsize: Figure size as a tuple (width, height)
    """
    # Create figure and axis
    fig, ax = plt.subplots(figsize=figsize)
    
    # Plot training points
    train_presence = train_gdf[train_gdf['Presence'] == 1]
    train_absence = train_gdf[train_gdf['Presence'] == 0]
    
    # Plot testing points
    test_presence = test_gdf[test_gdf['Presence'] == 1]
    test_absence = test_gdf[test_gdf['Presence'] == 0]
    
    # Plot each category with different colors and markers
    ax.scatter(train_presence.geometry.x, train_presence.geometry.y, 
               c='forestgreen', marker='o', s=50, alpha=0.7, label='Train - Presence')
    ax.scatter(train_absence.geometry.x, train_absence.geometry.y, 
               c='lightblue', marker='o', s=50, alpha=0.7, label='Train - Absence')
    ax.scatter(test_presence.geometry.x, test_presence.geometry.y, 
               c='darkgreen', marker='X', s=80, alpha=0.9, label='Test - Presence')
    ax.scatter(test_absence.geometry.x, test_absence.geometry.y, 
               c='darkblue', marker='X', s=80, alpha=0.9, label='Test - Absence')
    
    # Add title and labels
    ax.set_title('Train-Test Split Visualization', fontsize=16)
    ax.set_xlabel('Easting', fontsize=14)
    ax.set_ylabel('Northing', fontsize=14)
    
    # Add legend
    ax.legend(fontsize=12, loc='best')
    
    # Add grid
    ax.grid(True, alpha=0.3, linestyle='--')
    
    # Adjust layout
    plt.tight_layout()
    
    return fig, ax

## Tile Encoding Functions

This section contains functions for batch processing and encoding multiple tiles. These functions handle the bulk conversion of image data and provide progress tracking and error handling.

### `encode_tile`
Reads a GeoTIFF tile and encodes it as compressed bytes. The function:
- Loads the raster data using rasterio
- Preserves both pixel data and metadata
- Compresses the data using numpy's savez_compressed
- Returns the encoded data as bytes

### `encode_all_tiles`
Processes and encodes all tiles in the DataFrame. Key features:
- Uses tqdm for progress tracking
- Handles encoding errors gracefully
- Provides summary statistics of successful/failed encodings
- Returns a filtered DataFrame containing only successfully encoded tiles
- Includes batch processing capability for memory efficiency

In [ ]:
def encode_all_tiles(df: pd.DataFrame, batch_size: int = 100) -> pd.DataFrame:
    """Encode all tiles in the DataFrame."""
    result_df = df.copy()
    result_df["encoded_tile"] = None
    failed_encodings = []

    for idx in tqdm(df.index, desc="Encoding tiles"):
        try:
            result_df.loc[idx, "encoded_tile"] = encode_tile(df.loc[idx, "tile_path"])
        except (FileNotFoundError, ValueError) as e:
            failed_encodings.append((df.loc[idx, "tile_path"], str(e)))

    if failed_encodings:
        print(f"\nFailed to encode {len(failed_encodings)} tiles")

    success_mask = result_df["encoded_tile"].notna()
    print(f"\nSuccessfully encoded {success_mask.sum()} out of {len(df)} tiles")
    return result_df[success_mask].copy()

## Data Storage Functions

This section handles the efficient storage of processed data. The functions here manage the serialization of DataFrames into parquet format for use with Hugging Face datasets.

### `save_to_parquet`
Saves a GeoDataFrame to a single parquet file. Key features:
- Creates directory structure if it doesn't exist
- Converts geometry to WKT format for storage
- Saves the DataFrame as a parquet file
- Reports the number of rows saved

In [ ]:
def save_to_parquet(gdf, output_path, convert_geometry=True):
    """
    Save GeoDataFrame to a single parquet file.
    
    Args:
        gdf: GeoDataFrame to save
        output_path: Path to save the parquet file
        convert_geometry: Whether to convert geometry to WKT
    """
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Convert geometry to WKT for storage if needed
    df = gdf.copy()
    if convert_geometry and 'geometry' in df.columns:
        df['geometry'] = df['geometry'].apply(lambda geom: geom.wkt if geom else None)
    
    # Save to parquet
    df.to_parquet(output_path, index=True)
    print(f"Saved {len(df)} rows to {output_path}")

### `create_hf_documentation`
Creates Markdown documentation for the Hugging Face dataset. The documentation includes:
- Dataset description
- Split methodology
- Statistics about the train and test sets
- Information about the data files
- Usage examples

This documentation helps users understand the dataset structure and how to use it with the Hugging Face datasets library.

In [ ]:
def create_hf_documentation(train_gdf, test_gdf, train_indices, test_indices, output_path):
    """
    Create Markdown documentation for Hugging Face dataset.
    
    Args:
        train_gdf: Training GeoDataFrame
        test_gdf: Testing GeoDataFrame
        train_indices: List of training indices
        test_indices: List of testing indices
        output_path: Path to save the markdown file
    """
    # Create Markdown documentation
    hf_markdown = f"""
# MPG Ranch Horse Detection Dataset

## Dataset Description

This dataset contains aerial imagery tiles for horse presence/absence detection at MPG Ranch. 
The data has been split into training and testing sets according to specific criteria.

## Split Methodology

The dataset was split into training (80%) and testing (20%) sets using the following criteria:

1. **Random Seed**: {RANDOM_SEED} (for reproducibility)
2. **Absence Points**: 
   - Excluded any absence found within 2 meters of a presence (at any time point)
   - Grouped by spatial blocks created using K-means clustering
   - Sampled 20% randomly from each spatial block for testing
3. **Presence Points**:
   - Grouped by zone and period
   - Sampled 20% randomly from each group for testing

## Split Statistics

- **Total Points**: {len(train_gdf) + len(test_gdf)}
- **Training Set**: {len(train_gdf)} points ({len(train_gdf)/(len(train_gdf) + len(test_gdf))*100:.1f}%)
  - Presence: {len(train_gdf[train_gdf['Presence'] == 1])}
  - Absence: {len(train_gdf[train_gdf['Presence'] == 0])}
- **Testing Set**: {len(test_gdf)} points ({len(test_gdf)/(len(train_gdf) + len(test_gdf))*100:.1f}%)
  - Presence: {len(test_gdf[test_gdf['Presence'] == 1])}
  - Absence: {len(test_gdf[test_gdf['Presence'] == 0])}

## Data Files

```yaml
data_files:
- split: train
  path: "data/train.parquet"
- split: test
  path: "data/test.parquet"
```

## Usage

```python
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("path/to/dataset")

# Access the splits
train_data = dataset["train"]
test_data = dataset["test"]
```
"""
    
    # Save the markdown file
    with open(output_path, 'w') as f:
        f.write(hf_markdown)
    
    print(f"Hugging Face documentation created at {output_path}")

## Main Processing Pipeline

This section executes the complete data processing pipeline:
1. Loads and analyzes ground truth data
2. Creates the feature DataFrame
3. Creates a train-test split
4. Saves split information and creates documentation
5. Encodes tiles for both train and test sets
6. Saves the processed data in parquet format for Hugging Face

The pipeline includes comprehensive error handling and progress reporting at each step.

In [ ]:
if __name__ == "__main__":
    try:
        # Load and analyze ground truth data
        gdf = load_ground_truth()

        # Print basic information
        info = get_point_info(gdf)
        print("\nGround Truth Information:")
        for key, value in info.items():
            print(f"{key}: {value}")

        # First, filter absences near presences in the original ground truth data
        presence_gdf = gdf[gdf['Presence'] == 1].copy()
        valid_absences = filter_absences_near_presences(gdf)
        
        # Create spatial blocks for absences
        absence_blocks = create_spatial_blocks(valid_absences)
        
        # Initialize lists to store indices
        presence_test_indices = []
        presence_train_indices = []
        absence_test_indices = []
        absence_train_indices = []
        
        # Split presences by zone and period
        test_size = 0.2
        for (zone, period), group in presence_gdf.groupby(['Zone', 'Period']):
            if len(group) > 0:
                # Calculate number of test samples for this group
                n_test = max(1, int(len(group) * test_size))
                
                # Randomly select indices for test set
                group_indices = group.index.tolist()
                test_idx = random.sample(group_indices, n_test)
                train_idx = [idx for idx in group_indices if idx not in test_idx]
                
                # Add to master lists
                presence_test_indices.extend(test_idx)
                presence_train_indices.extend(train_idx)
        
        # Split absences by spatial block
        for block, group in absence_blocks.groupby('spatial_block'):
            if len(group) > 0:
                # Calculate number of test samples for this block
                n_test = max(1, int(len(group) * test_size))
                
                # Randomly select indices for test set
                group_indices = group.index.tolist()
                test_idx = random.sample(group_indices, n_test)
                train_idx = [idx for idx in group_indices if idx not in test_idx]
                
                # Add to master lists
                absence_test_indices.extend(test_idx)
                absence_train_indices.extend(train_idx)
        
        # Combine test and train indices
        gt_test_indices = presence_test_indices + absence_test_indices
        gt_train_indices = presence_train_indices + absence_train_indices
        
        # Create train and test GeoDataFrames from the original ground truth data
        gt_test_gdf = gdf.loc[gt_test_indices].copy()
        gt_train_gdf = gdf.loc[gt_train_indices].copy()
        
        # Print split statistics for ground truth data
        print(f"Total ground truth points: {len(gdf)}")
        print(f"Training points: {len(gt_train_gdf)} ({len(gt_train_gdf)/len(gdf)*100:.1f}%)")
        print(f"Testing points: {len(gt_test_gdf)} ({len(gt_test_gdf)/len(gdf)*100:.1f}%)")
        
        # Save the ground truth GeoDataFrames as GeoJSON for reference
        gt_train_gdf.to_file(os.path.join(SPLITS_DIR, 'train_split.geojson'), driver='GeoJSON')
        gt_test_gdf.to_file(os.path.join(SPLITS_DIR, 'test_split.geojson'), driver='GeoJSON')
        
        # Visualize the train-test split on the ground truth data
        print("\nVisualizing train-test split...")
        fig, ax = visualize_train_test_split(gt_train_gdf, gt_test_gdf)
        plt.show()
        
        # Now create feature DataFrames for train and test sets separately
        print("\nCreating feature DataFrames...")
        train_feature_gdf = create_feature_dataframe(gt_train_gdf)
        test_feature_gdf = create_feature_dataframe(gt_test_gdf)
        
        # Print feature DataFrame info
        print("\nTrain Feature DataFrame Info:")
        print(train_feature_gdf.info())
        print("\nTest Feature DataFrame Info:")
        print(test_feature_gdf.info())
        
        # Save the indices as JSON for reference
        split_indices = {
            'train_indices': gt_train_indices,
            'test_indices': gt_test_indices,
            'metadata': {
                'random_seed': RANDOM_SEED,
                'train_size': len(gt_train_indices),
                'test_size': len(gt_test_indices),
                'train_presence_count': len(gt_train_gdf[gt_train_gdf['Presence'] == 1]),
                'train_absence_count': len(gt_train_gdf[gt_train_gdf['Presence'] == 0]),
                'test_presence_count': len(gt_test_gdf[gt_test_gdf['Presence'] == 1]),
                'test_absence_count': len(gt_test_gdf[gt_test_gdf['Presence'] == 0]),
            }
        }
        
        with open(os.path.join(SPLITS_DIR, 'split_indices.json'), 'w') as f:
            json.dump(split_indices, f, indent=2)
        
        # Create Hugging Face documentation
        create_hf_documentation(
            gt_train_gdf, 
            gt_test_gdf, 
            gt_train_indices, 
            gt_test_indices, 
            os.path.join(HF_DATA_DIR, 'README.md')
        )

        # Process and encode tiles for training set
        print("\nEncoding training tiles...")
        train_encoded_df = encode_all_tiles(train_feature_gdf)
        
        # Process and encode tiles for testing set
        print("\nEncoding testing tiles...")
        test_encoded_df = encode_all_tiles(test_feature_gdf)
        
        # Save to parquet files in the format required by Hugging Face
        save_to_parquet(train_encoded_df, os.path.join(HF_DATA_DIR, 'train.parquet'))
        save_to_parquet(test_encoded_df, os.path.join(HF_DATA_DIR, 'test.parquet'))
        
        print("\nProcessing complete. Dataset is ready for Hugging Face.")

    except Exception as e:
        print(f"Error: {str(e)}")
        import traceback
        traceback.print_exc()